In [276]:
!pip install beautifulsoup4
!pip install requests
!pip install pandas
!pip install tqdm
!pip install scikit-learn
!pip install matplotlib
!pip install seaborn

  Using cached cycler-0.12.1-py3-none-any.whl.metadata (3.8 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.1/8.1 MB 44.6 MB/s eta 0:00:00
Using cached cycler-0.12.1-py3-none-any.whl (8.3 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 34.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 38.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7/7 [matplotlib]7 [matplotlib]


In [136]:
# 요약보기
pd.set_option('display.max_rows', 100)

In [132]:
#전체행 보기 
pd.set_option('display.max_rows', None)

In [277]:
%matplotlib inline

In [278]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from tqdm import tqdm
import re
from datetime import datetime, timedelta
import zipfile
import io
import xml.etree.ElementTree as ET
from concurrent.futures import ThreadPoolExecutor, as_completed
from datetime import datetime, timedelta
import pandas as pd
from tqdm import tqdm
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.preprocessing import StandardScaler
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# 저평가 중소형주 발굴을 통한 투자 전략 수립

## 1. 중소형주 후보군 선정
### 1) 선정기준
- 전체 상장 종목(코스피 + 코스닥) 중 시가총액 하위 30% 추출
- 중소형주 중 최근 1년 수익률 상위 100개 추출

### 2) 시가 총액 하위 30% ~ 70% 추출
#### - 한국거래소 API에서 시가 총액 기준으로 크롤링

In [44]:
def get_stock_info(basDd):
  url = {
  "kospi_url": "http://data-dbg.krx.co.kr/svc/apis/sto/stk_bydd_trd",
  "kosdaq_url": "http://data-dbg.krx.co.kr/svc/apis/sto/ksq_bydd_trd"
  }

  params = {
    "basDd": basDd.strftime("%Y%m%d")
  }

  headers = {
    "AUTH_KEY": "18CEC6172A764513B4C60CAA3E3186D3AE2453C3"
  }
  market_is_open = requests.get(list(url.values())[0], params=params, headers=headers)
  if (market_is_open.json()['OutBlock_1'] == []):
    # print("Market is closed")
    return None
  # while (market_is_open.json()['OutBlock_1'] == []):
  #   print("Market is closed")
  #   basDd = basDd + timedelta(days=1)
  #   params['basDd'] = basDd.strftime("%Y%m%d")
  #   market_is_open = requests.get(list(url.values())[0], params=params, headers=headers)

  all_stock_info = pd.DataFrame()
    
  for url in url.values():
    response = requests.get(url, params=params, headers=headers)
    if response.status_code == 200:
      data = response.json()
      if 'OutBlock_1' in data:
          krx_stock_info = pd.DataFrame(data['OutBlock_1'])

          # 컬럼명 한글로 변환
          column_map = {
              'BAS_DD': '기준일자',
              'ISU_CD': '종목코드',
              'ISU_NM': '종목명',
              'MKT_NM': '시장구분',
              'SECT_TP_NM': '소속부',
              'TDD_CLSPRC': '종가',
              'CMPPREVDD_PRC': '대비',
              'FLUC_RT': '등락률',
              'TDD_OPNPRC': '시가',
              'TDD_HGPRC': '고가',
              'TDD_LWPRC': '저가',
              'ACC_TRDVOL': '거래량',
              'ACC_TRDVAL': '거래대금',
              'MKTCAP': '시가총액',
              'LIST_SHRS': '상장주식수'
          }

          krx_stock_info = krx_stock_info.rename(columns=column_map)

          numeric_columns = ['종가', '대비', '등락률', '시가', '고가', '저가', '거래량', '거래대금', '시가총액', '상장주식수']
          
          for col in numeric_columns:
              if col in krx_stock_info.columns:
                  krx_stock_info[col] = pd.to_numeric(krx_stock_info[col].str.replace(',', ''), errors='coerce')
          
          all_stock_info = pd.concat([all_stock_info, krx_stock_info])

      # print("Data retrieved successfully:")
    else:
        print(f"Failed to retrieve data. HTTP Status Code: {response.status_code}")
        print("Response:", response.text)
  return all_stock_info

In [10]:
basDd = datetime.strptime("20250103", "%Y%m%d")
# basDd = datetime.now() - timedelta(days=1)
all_stock_info = get_stock_info(basDd)
sorted_data = all_stock_info.sort_values(by='시가총액', ascending=True)

Data retrieved successfully:
Data retrieved successfully:


#### - ETF, ETN, 지수추종, 우선주, 관리종목 등 종목 필터링

In [11]:
# ETF 관련 키워드
filter_ETF = [
    "TIGER", "KODEX", "HANARO", "ARIRANG", 
    "KBSTAR", "KOSEF", "SOL", "BNK", 
    "PLUS", "WON", "FOCUS", "KIWOOM", "ACE",
    "KoAct", "액티브", "S&P", "TREX", "ITF", "나스닥",
    "밸류업", "(H)", "마이티", "코스피"
]

# 인버스/레버리지 관련 키워드
filter_inverse_leverage = [
    "인버스", "레버리지", "선물", "ETN", "RISE"
]

# 우선주/전환사채 관련 키워드
filter_preferred = [
    "우선주", "우B", "전환사채"
]

# 기타 제외할 키워드
filter_etc = [
    "스팩", "리츠", "배당"
]

filter_sector = [
  '관리종목(소속부없음)', '투자주의환기종목(소속부없음)', '외국기업(소속부없음)'
]

# 전체 필터링 키워드 통합
filter_keyword = list(set(
    filter_ETF + 
    filter_inverse_leverage + 
    filter_preferred + 
    filter_etc
))


keyword_filtered = sorted_data[~sorted_data['종목명'].apply(lambda x: 
    any(keyword in x for keyword in filter_keyword) or 
    bool(re.search(r'우$', x)) or
    bool(re.search(r'^HK', x))

)]

sector_filtered = keyword_filtered[~keyword_filtered['소속부'].apply(lambda x : x in filter_sector)]

# 하위 30%의 종목만 필터링
filtered_data = sector_filtered.iloc[int(len(sorted_data)*0.4):int(len(sorted_data)*0.8)]
print(len(filtered_data))

1098


#### - 거래량 필터링

In [12]:
# 거래량 기본 통계 확인
filtered_data = filtered_data[filtered_data['거래량'].apply(lambda x : x != 0)]

volume_stats = {
    '최대 거래량': filtered_data['거래량'].max(),
    '최소 거래량': filtered_data['거래량'].min(),
    '평균 거래량': filtered_data['거래량'].mean(),
    '중간값': filtered_data['거래량'].median(),
    '표준편차': filtered_data['거래량'].std()
}

# 결과 출력
for key, value in volume_stats.items():
    print(f"{key}: {value:,.0f}")

# 거래량 분포 확인 (사분위수)
print("\n사분위수 분포:")
print(filtered_data['거래량'].describe())

# 중앙값 이상이면서 상위 95%에 해당하는 종목 필터링
volume_filtered = filtered_data[
    (filtered_data['거래량'] >= filtered_data['거래량'].quantile(0.5)) &
    (filtered_data['거래량'] <= filtered_data['거래량'].quantile(0.95)) 
]
print(len(volume_filtered))

최대 거래량: 66,692,558
최소 거래량: 121
평균 거래량: 641,519
중간값: 95,224
표준편차: 2,902,748

사분위수 분포:
count    1.089000e+03
mean     6.415189e+05
std      2.902748e+06
min      1.210000e+02
25%      3.396400e+04
50%      9.522400e+04
75%      3.165080e+05
max      6.669256e+07
Name: 거래량, dtype: float64
490


In [13]:
last_year = basDd - timedelta(days=365)
last_stock_info = get_stock_info(last_year)

Data retrieved successfully:
Data retrieved successfully:


In [14]:
# 현재 데이터와 작년 데이터를 종목명 기준으로 합치기
merged_df = volume_filtered.merge(last_stock_info, 
            on='종목명',  # 종목명을 기준으로 합침
            how='left',   # 현재 데이터(365종목)를 기준으로 left join
            suffixes=('_current', '_last_year'))  # 컬럼 구분을 위한 접미사
merged_df["연간수익률"] = (merged_df["종가_current"] - merged_df["종가_last_year"]) / merged_df["종가_last_year"]
merged_df["시총증감률"] = (merged_df["시가총액_current"] - merged_df["시가총액_last_year"]) / merged_df["시가총액_last_year"]
print(len(merged_df))
merged_df['연간수익률'].describe()
merged_df['시총증감률'].describe()

490


count    448.000000
mean       0.021187
std        0.681786
min       -0.779721
25%       -0.316009
50%       -0.139108
75%        0.135856
max        6.812737
Name: 시총증감률, dtype: float64

### 3) 구간별 수익률 vs 시총증감률 결과 분석

#### 주가 수익률 기준

| 구간 | 종목수 | 평균 | 표준편차 | 최소값 | 중간값 | 최대값 |
|------|--------|------|----------|--------|--------|--------|
| ~30% | 332개 | -3.38% | 73.95% | -79.15% | -12.34% | 924.77% |
| 30~70% | 445개 | 6.93% | 57.26% | -77.91% | -1.48% | 735.28% |
| 50~80% | 340개 | 16.62% | 72.39% | -77.91% | 2.02% | 702.27% |
| 40~80% | 456개 | 13.51% | 66.04% | -77.91% | 0.62% | 702.27% |

#### 시총증감률 기준

| 구간 | 종목수 | 평균 | 표준편차 | 최소값 | 중간값 | 최대값 |
|------|--------|------|----------|--------|--------|--------|
| ~30% | 332개 | -9.57% | 28.06% | -79.15% | -11.49% | 129.20% |
| 30~70% | 445개 | 9.63% | 45.94% | -77.82% | 1.34% | 304.15% |
| 50~80% | 340개 | 21.80% | 74.88% | -77.82% | 3.66% | 725.30% |
| 40~80% | 456개 | 18.17% | 68.49% | -77.82% | 2.71% | 725.30% |


### 4) 종목 최종 선정
- 시총 증감률 기준 선택 : 주가 + 상장주식수 변동을 모두 반영한 실제 기업가치 변화
- 시총 40% ~ 80% 구간 선정 : 4개의 표본 중 중간 표준편차, 안정적 수익 기대

In [20]:
selected_df = merged_df[
    (merged_df['시총증감률'] >= merged_df['시총증감률'].quantile(0.6)) &  
    (merged_df['시총증감률'] <= merged_df['시총증감률'].quantile(0.9))  
]

selected_df['시총증감률'].describe()

count    134.000000
mean       0.186527
std        0.189559
min       -0.050064
25%        0.026579
50%        0.135938
75%        0.330584
max        0.584381
Name: 시총증감률, dtype: float64

In [21]:
final_df = selected_df.filter(
  items=['시장구분_current', '종목명', '종목코드_current', '소속부_current', '시총증감률', '연간수익률', '상장주식수_current']
  ).rename(columns={
    '시장구분_current': '시장구분',
    '종목코드_current': '종목코드',
    '소속부_current': '소속부',
    '시총증감률': '시총_증감률',
    '연간수익률': '주가_수익률',
    '상장주식수_current': '상장주식수'
})

## 2. 재무재표 정보 수집
### 1) 기업코드 가져오기

In [22]:
url = "https://opendart.fss.or.kr/api/corpCode.xml"

params = {
    "crtfc_key": "4758096e577a50f35eaec98b171e463367c7c23f"
}

# ZIP 파일 다운로드
response = requests.get(url, params=params)

# ZIP 파일 압축 해제
with zipfile.ZipFile(io.BytesIO(response.content)) as zf:
    # CORPCODE.xml 파일 읽기
    xml_data = zf.read('CORPCODE.xml')
    
# XML 파싱
root = ET.fromstring(xml_data)

# 데이터 확인 (예시)
data = []
for company in root.findall('list'):
    stock_code = company.findtext('stock_code')
    if stock_code and stock_code.strip():  # 종목코드가 존재하고 공백이 아닌 경우만
        data.append({
            '고유번호': company.findtext('corp_code'),
            '회사명': company.findtext('corp_name'),
            '종목코드': stock_code,
            '최종변경일자': company.findtext('modify_date')
        })

# DataFrame 생성
analysis_df = pd.DataFrame(data)

In [23]:
final_summary = pd.merge(final_df, 
                  analysis_df[['고유번호', '종목코드']],
                  on='종목코드',
                  how='left')

### 2) 재무재표 가져오기(1년)

In [246]:
def api_call(corp_code, bsns_year, reprt_code):
   url = "https://opendart.fss.or.kr/api/fnlttSinglAcntAll.json"
   params = {
       "crtfc_key": "4758096e577a50f35eaec98b171e463367c7c23f",
       "corp_code": corp_code,
       "bsns_year": bsns_year,
       "reprt_code": reprt_code,
       "fs_div": "OFS"
   }
   response = requests.get(url, params=params)
   return response

def find_account_by_keywords(df, keyword_list):
   """키워드 리스트로 계정과목 찾기"""
   for keyword in keyword_list:
       found = df[df['account_nm'] == keyword]
       if not found.empty:
           return found.iloc[0]
   return None

def get_quarter_data(response):
   """한 분기 데이터를 딕셔너리로 반환"""
   df = pd.DataFrame(response.json()['list'])

   disclosure_date = df.iloc[0]['rcept_no'][:8] if not df.empty else None
   
   account_keywords = {
      'CIS': {
        '매출액': ['매출액', '수익(매출액)', 'I. 영업수익','영업수익','수익 합계','Ⅰ. 매출액'],
        '매출총이익': ['매출총이익', 'III. 영업이익(손실)','Ⅲ. 매출총이익(손실)'],  # 영업수익-영업비용
        '매출원가': ['매출원가', 'Ⅱ. 영업비용'],
        '영업이익': ['영업이익(손실)', '영업이익', 'III. 영업이익(손실)','Ⅳ. 영업이익(손실)','영업손익'],
        '당기순이익': ['당기순이익(손실)', 'VIII. 분기순이익(손실)',
                   'VIII. 당기순이익(손실)','당기순이익',
                   '분기순이익','반기순이익','분기순이익(손실)','반기순이익(손실)','연결분기순이익','연결반기순이익','연결당기순이익',
                   '1. 당기순이익(손실)','당기연결순손익', '반기순손익','연결반기순이익(손실)','연결분기순이익(손실)','연결분기순이익'],
        '금융원가': ['금융원가', '금융비용', '이자비용'],
        '금융수익': ['금융수익', '금융수입', '이자수익']
    },
       'BS_자산': {
           '자산총계': ['자산총계','기말자산','자산'],
           '유동자산': ['유동자산'],
           '현금및현금성자산': ['현금및현금성자산', '현금성자산']
       },
       'BS_부채': {
           '부채총계': ['부채총계','기말부채','부채'],
           '유동부채': ['유동부채']
       },
       'BS_자본': {
           '자본총계': ['자본총계','기말자본','자본'],
           '자본금': ['자본금']
       }
   }
   
   quarter_data = {}
   
   for category, account_dict in account_keywords.items():
       for account_key, keyword_list in account_dict.items():
           item = find_account_by_keywords(df, keyword_list)
           if item is not None:
               key = f"{category}_{account_key}"
               amount = item['thstrm_amount']
               if pd.notna(amount) and amount != '':
                   quarter_data[key] = int(amount)
               else:
                   quarter_data[key] = 0
           else:
               key = f"{category}_{account_key}"
               quarter_data[key] = 0
   
   return quarter_data, disclosure_date

def get_corp_quarter_data(corp_code, bsns_year, reprt_code = None):
    # 4개 분기 API 호출 및 데이터 수집
    response_2023_4q = api_call(corp_code, str(int(bsns_year)-1), "11011")
    response_2024_1q = api_call(corp_code, bsns_year, "11013")
    response_2024_2q = api_call(corp_code, bsns_year, "11012")
    response_2024_3q = api_call(corp_code, bsns_year, "11014")
    response_2024_4q = api_call(corp_code, bsns_year, "11011")

    q4_2023, q4_2023_date = get_quarter_data(response_2023_4q)
    q1_2024, q1_date = get_quarter_data(response_2024_1q)
    q2_2024, q2_date = get_quarter_data(response_2024_2q)
    q3_2024, q3_date = get_quarter_data(response_2024_3q)
    q4_2024, q4_date = get_quarter_data(response_2024_4q)

    # 통합 테이블 생성
    results = []

    results.append({
        '구분': '공시정보',
        '계정과목': '공시날짜',
        '2023_4Q': q4_2023_date if q4_2023_date else 'N/A',
        '2024_1Q': q1_date if q1_date else 'N/A',
        '2024_2Q': q2_date if q2_date else 'N/A', 
        '2024_3Q': q3_date if q3_date else 'N/A',
        '2024_4Q': q4_date if q4_date else 'N/A',
        '발견여부': 'O'
    })

    account_list = {
        'CIS': ['매출액', '매출총이익', '매출원가', '영업이익', '당기순이익', '금융원가', '금융수익'],
        'BS_자산': ['자산총계', '유동자산', '현금및현금성자산'],
        'BS_부채': ['부채총계', '유동부채'],
        'BS_자본': ['자본총계', '자본금']
    }

    for category, accounts in account_list.items():
        for account in accounts:
            key = f"{category}_{account}"
            
            results.append({
                '구분': category,
                '계정과목': account,
                '2023_4Q': q4_2023.get(key, 0),
                '2024_1Q': q1_2024.get(key, 0),
                '2024_2Q': q2_2024.get(key, 0),
                '2024_3Q': q3_2024.get(key, 0),
                '2024_4Q': q4_2024.get(key, 0),
                '발견여부': 'O' if q1_2024.get(key, 0) != 0 else 'X'
            })

    # DataFrame 생성 및 출력
    result_df = pd.DataFrame(results)
    return result_df

In [53]:
from concurrent.futures import ThreadPoolExecutor

def background_task(corp_code, corp_name):
    data = get_corp_quarter_data(corp_code, "2024")
    return corp_name, data

with ThreadPoolExecutor(max_workers=10) as executor:
    futures = [
        executor.submit(background_task, row['고유번호'], row['종목명']) 
        for _, row in final_summary.iterrows()
    ]
    results = [future.result() for future in futures]

# 딕셔너리로 변환
result_dict = {name: df for name, df in results}
print(result_dict)

{'제이스코홀딩스':        구분      계정과목       2023_4Q       2024_1Q       2024_2Q       2024_3Q  \
0    공시정보      공시날짜      20240718      20240516      20240813      20241114   
1     CIS       매출액   55349435845    7585645662    7055258448    6140077528   
2     CIS     매출총이익             0    -456291285   -1004865370   -1318062478   
3     CIS      매출원가   56043349499    8041936947    8060123818    7458140006   
4     CIS      영업이익   -8345870361   -2144632392   -4522977551   -4216707610   
5     CIS     당기순이익  -24436001211   -4677021476    3366528375   -6304108419   
6     CIS      금융원가   15106004335    2356064598    1968933230    1842232460   
7     CIS      금융수익   12679396449     446635210    9025739710    -543493706   
8   BS_자산      자산총계  128018104109  124598129193  126197378478  125269453566   
9   BS_자산      유동자산   31863453708   29924467098   37763355133   36342750517   
10  BS_자산  현금및현금성자산    1291491475      45832944     699424600     414603166   
11  BS_부채      부채총계   65143940538   6209

### 3) 주식정보 가져오기(1년)

In [ ]:
# start_date = datetime(2024, 1, 1)
# end_date = datetime(2024, 12, 31)

# current_date = start_date
# all_data = pd.DataFrame()

# while current_date <= end_date:
#     stock_data = get_stock_info(current_date)
#     if not stock_data is None:
#         temp_data = stock_data.filter(items=['종목코드', '종목명', '종가'])
#         # 현재 날짜를 컬럼명으로 사용
#         date_str = current_date.strftime("%Y%m%d")
#         temp_data = temp_data.rename(columns={'종가': date_str})
        
#         # 처음이면 그대로, 아니면 종목코드와 종목명 기준으로 merge
#         if all_data.empty:
#             all_data = temp_data
#         else:
#             all_data = pd.merge(all_data, temp_data, on=['종목코드', '종목명'], how='outer')
    
#     current_date += timedelta(days=1)
# all_data.to_csv('all_data.csv', index=False)

In [ ]:
def get_daily_data(date):
    """단일 날짜의 데이터 가져오기"""
    stock_data = get_stock_info(date)
    if stock_data is not None:
        temp_data = stock_data.filter(items=['종목코드', '종목명', '종가'])
        date_str = date.strftime("%Y%m%d")
        temp_data = temp_data.rename(columns={'종가': date_str})
        return date, temp_data
    return date, None

# 날짜 리스트 생성
start_date = datetime(2024, 1, 1)
end_date = datetime(2024, 12, 31)
date_list = []
current_date = start_date
while current_date <= end_date:
    date_list.append(current_date)
    current_date += timedelta(days=1)

# 병렬 처리 with 진행상황
results = []
with ThreadPoolExecutor(max_workers=10) as executor:
    futures = {executor.submit(get_daily_data, date): date for date in date_list}
    
    for future in tqdm(as_completed(futures), total=len(futures), desc="데이터 수집"):
        results.append(future.result())

# 날짜순으로 정렬하고 데이터 병합
results.sort(key=lambda x: x[0])  # 날짜 기준 정렬
all_data = pd.DataFrame()

for date, temp_data in results:
    if temp_data is not None:
        if all_data.empty:
            all_data = temp_data
        else:
            all_data = pd.merge(all_data, temp_data, on=['종목코드', '종목명'], how='outer')

all_data.to_csv('all_data.csv', index=False)

In [34]:
all_data = pd.read_csv('all_data.csv')

In [37]:
sample = pd.merge(final_summary, all_data, on=['종목코드', '종목명'], how='inner')

In [50]:
result_dict

{'제이스코홀딩스':        구분      계정과목       2023_4Q       2024_1Q       2024_2Q       2024_3Q  \
 0    공시정보      공시날짜      20240718      20240516      20240813      20241114   
 1     CIS       매출액   55349435845    7585645662    7055258448    6140077528   
 2     CIS     매출총이익             0    -456291285   -1004865370   -1318062478   
 3     CIS      매출원가   56043349499    8041936947    8060123818    7458140006   
 4     CIS      영업이익   -8345870361   -2144632392   -4522977551   -4216707610   
 5     CIS     당기순이익  -24436001211   -4677021476    3366528375   -6304108419   
 6     CIS      금융원가   15106004335    2356064598    1968933230    1842232460   
 7     CIS      금융수익   12679396449     446635210    9025739710    -543493706   
 8   BS_자산      자산총계  128018104109  124598129193  126197378478  125269453566   
 9   BS_자산      유동자산   31863453708   29924467098   37763355133   36342750517   
 10  BS_자산  현금및현금성자산    1291491475      45832944     699424600     414603166   
 11  BS_부채      부채총계   651439

In [46]:
sample.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 134 entries, 0 to 133
Columns: 252 entries, 시장구분 to 20241230
dtypes: float64(246), int64(1), object(5)
memory usage: 263.9+ KB


In [292]:
def get_account_value_by_name(financial_data, account_name, quarter_column):
    """계정과목 이름으로 값 찾기"""
    # 해당 계정과목이 있는 행 찾기
    mask = financial_data['계정과목'] == account_name
    account_row = financial_data[mask]
    
    if not account_row.empty:
        return int(account_row[quarter_column].iloc[0])
    return 0

def select_quarter_column(financial_data, date):
    date_int = int(date)
    for key, value in financial_data.iloc[0].items():
        if value not in ['공시정보', '공시날짜', 'O']:
            if date_int < int(value):
                print(key, value)
                return key

def get_corp_cal_index(financial_data, date, shares_outstanding, value):
  # 분기데이터 행 선택
    ratios = {}

    quarter = select_quarter_column(financial_data, date)

    annualization_factors = {
            '1Q': 4,      # 1분기 → 연간 추정
            '2Q': 2,      # 반기 → 연간 추정  
            '3Q': 4/3,    # 3분기 → 연간 추정
            '4Q': 1       # 연간 실제
        }

    factor = annualization_factors.get(quarter[-2:], 1)


    net_income_raw = financial_data[financial_data['계정과목'] == '당기순이익'][quarter].iloc[0]
    total_assets = financial_data[financial_data['계정과목'] == '자산총계'][quarter].iloc[0]
    total_equity = financial_data[financial_data['계정과목'] == '자본총계'][quarter].iloc[0]
    total_debt = financial_data[financial_data['계정과목'] == '부채총계'][quarter].iloc[0]
    revenue_raw = financial_data[financial_data['계정과목'] == '매출액'][quarter].iloc[0]
    operating_income_raw = financial_data[financial_data['계정과목'] == '영업이익'][quarter].iloc[0]

    print(net_income_raw, total_assets, total_equity, total_debt, revenue_raw, operating_income_raw)
    print(type(net_income_raw), type(total_assets), type(total_equity), type(total_debt), type(revenue_raw), type(operating_income_raw))

    net_income = net_income_raw * factor
    revenue = revenue_raw * factor  
    operating_income = operating_income_raw * factor

    eps = net_income / shares_outstanding if shares_outstanding > 0 else 0
    bps = total_equity / shares_outstanding if shares_outstanding > 0 else 0


    print(eps, bps, value)

    try:
        print("try =================================================")
        # 1. PER = 주가 / 주당순이익 (연간화된 EPS 사용)
        ratios['PER'] = value / eps if eps > 0 else None
        # 2. PBR = 주가 / 주당순자산 (재무상태표는 그대로)
        ratios['PBR'] = value / bps if bps > 0 else None
        
        # 3. ROE = 당기순이익 / 자기자본 (연간화된 순이익 사용)
        ratios['ROE'] = (net_income / total_equity * 100) if total_equity > 0 else None
        
        # 4. ROA = 당기순이익 / 총자산 (연간화된 순이익 사용)
        ratios['ROA'] = (net_income / total_assets * 100) if total_assets > 0 else None
        
        # 5. 영업이익률 = 영업이익 / 매출액 (둘 다 연간화되므로 비율은 동일)
        ratios['영업이익률'] = (operating_income_raw / revenue_raw * 100) if revenue_raw > 0 else None
        
        # 6. 부채비율 = 부채총계 / 자기자본 (재무상태표는 그대로)
        ratios['부채비율'] = (total_debt / total_equity * 100) if total_equity > 0 else None

        print(ratios)
        return ratios
    
    except Exception as e:
        print(e)
        print("error =================================================")
        return None
    
    

result22 = {}
  
for row in sample.itertuples():
    corp_code = None
    corp_name = None
    financial_data = None
    shares_outstanding = None

    first_row = sample.iloc[0]
    # row[1:]
    for column_name, value in zip(sample.columns, first_row):
        # print(column_name, value)
        if column_name == '고유번호':
            corp_code = value
            financial_data = get_corp_quarter_data(corp_code, "2024")
        elif column_name == "종목명":
            corp_name = value
        elif column_name == "상장주식수":
            shares_outstanding = value
        elif re.match(r'^202', column_name):
            result22 = get_corp_cal_index(financial_data, column_name, shares_outstanding, value)
            print(result22)

KeyboardInterrupt: 

In [279]:
print(result22)

None


In [262]:
import re

for row in sample.itertuples():
    corp_code = None
    corp_name = None
    financial_data = None
    shares_outstanding = None

    first_row = sample.iloc[0]
    # row[1:]
    for column_name, value in zip(sample.columns, first_row):
        # print(column_name, value)
        if column_name == '고유번호':
            corp_code = value
            financial_data = get_corp_quarter_data(corp_code, "2024")
        elif column_name == "종목명":
            corp_name = value
        elif column_name == "상장주식수":
            shares_outstanding = value
        elif re.match(r'^202', column_name):
            result22 = get_corp_cal_index(financial_data, column_name, shares_outstanding, value)

In [263]:
result22

{'PER': None,
 'PBR': None,
 'ROE': None,
 'ROA': None,
 '영업이익률': None,
 '부채비율': None}

sample


In [115]:
stock_data = get_stock_info(datetime.strptime("20250101", "%Y%m%d"))
stock_data.filter(items=['종가'])

Market is closed
Market is closed
Data retrieved successfully:
Data retrieved successfully:


,종가
0,4305
1,9640
2,3450
3,102500
4,10340
...,...
1779,12490
1780,5320
1781,1620
1782,4960


In [111]:
test_result_1 = get_corp_quarter_data(result['고유번호'].iloc[134], "2024")
test_result_1

,구분,계정과목,2024_1Q,2024_2Q,2024_3Q,2024_4Q,발견여부
0,CIS,매출액,56573739300,97327119861,147192704045,409378252326,O
1,CIS,매출총이익,33284345957,67101020345,88094272459,248980684761,O
2,CIS,매출원가,23289393343,30226099516,59098431586,160397567565,O
3,CIS,영업이익,0,36508391638,0,98121004725,X
4,CIS,당기순이익,0,0,0,107832255114,X
5,CIS,금융원가,913227354,917024247,909827130,3644086629,O
6,CIS,금융수익,152414463,162747810,1186249488,3886832701,O
7,BS_자산,자산총계,0,0,0,985122548000,X
8,BS_자산,유동자산,320501725900,373058510871,372219136415,392448612730,O
9,BS_자산,현금및현금성자산,169044933891,210176740407,222463025873,246023909099,O


In [94]:
url = "https://opendart.fss.or.kr/api/fnlttSinglAcntAll.json"
params = {
    "crtfc_key": "4758096e577a50f35eaec98b171e463367c7c23f",
    "corp_code": "00117601",
    "bsns_year": "2024",
    "reprt_code": "11013",
    "fs_div": "OFS"
}

response = requests.get(url, params=params)
data = response.json()
df = pd.DataFrame(response.json()['list'])
revenue_accounts = df[df['account_nm'].str.contains('매출', na=False)]['account_nm'].unique()
print("매출 관련 계정과목들:", revenue_accounts)

,rcept_no,reprt_code,bsns_year,corp_code,sj_div,sj_nm,account_id,account_nm,account_detail,thstrm_nm,thstrm_amount,frmtrm_nm,frmtrm_amount,ord,currency,thstrm_add_amount,frmtrm_q_nm,frmtrm_q_amount,frmtrm_add_amount
0,20240514000148,11013,2024,00117601,BS,재무상태표,dart_CashAndDuefromBanks,I. 현금및예치금,-,제 64 기 1분기말,1853380389000,제 63 기말,1789956709000,7,KRW,NaN,NaN,NaN,NaN
1,20240514000148,11013,2024,00117601,BS,재무상태표,-표준계정코드 미사용-,예치금,-,제 64 기 1분기말,1145402869000,제 63 기말,1149592530000,8,KRW,NaN,NaN,NaN,NaN
2,20240514000148,11013,2024,00117601,BS,재무상태표,ifrs-full_CashAndCashEquivalents,현금및현금성자산,-,제 64 기 1분기말,707977520000,제 63 기말,640364179000,9,KRW,NaN,NaN,NaN,NaN
3,20240514000148,11013,2024,00117601,BS,재무상태표,dart_LoansAtAmortisedCost,VI. 대출채권(상각후원가),-,제 64 기 1분기말,2599374184000,제 63 기말,2497284405000,10,KRW,NaN,NaN,NaN,NaN
4,20240514000148,11013,2024,00117601,BS,재무상태표,ifrs-full_Assets,자산총계,-,제 64 기 1분기말,15472894208000,제 63 기말,15998946579000,11,KRW,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
337,20240514000148,11013,2024,00117601,SCE,자본변동표,ifrs-full_Equity,자본총계,자본 [구성요소]|기타포괄손익누적액 [구성요소],제 64 기 1분기,135692572000,NaN,NaN,27,KRW,NaN,제 63 기 1분기,129328191000,NaN
338,20240514000148,11013,2024,00117601,SCE,자본변동표,ifrs-full_Equity,자본총계,자본 [구성요소]|이익잉여금 [구성요소],제 64 기 1분기,426346078000,NaN,NaN,27,KRW,NaN,제 63 기 1분기,372600876000,NaN
339,20240514000148,11013,2024,00117601,SCE,자본변동표,ifrs-full_Equity,자본총계,자본 [구성요소]|자본금 [구성요소],제 64 기 1분기,1062543940000,NaN,NaN,27,KRW,NaN,제 63 기 1분기,1062543940000,NaN
340,20240514000148,11013,2024,00117601,SCE,자본변동표,ifrs-full_Equity,자본총계,자본 [구성요소]|자본잉여금 [구성요소],제 64 기 1분기,1303000,NaN,NaN,27,KRW,NaN,제 63 기 1분기,1303000,NaN


## 3. 지표 생성
### 1) per, pbr, roe, roa, 영업이익률, 부채비율 생성

In [ ]:
def get_account_value_by_name(financial_data, account_name, quarter_column):
    """계정과목 이름으로 값 찾기"""
    mask = financial_data['계정과목'] == account_name
    account_row = financial_data[mask]
    
    if not account_row.empty:
        try:
            return float(account_row[quarter_column].iloc[0])
        except:
            return 0
    return 0

def select_quarter_column(financial_data, date):
    """해당 날짜에 맞는 분기 컬럼 선택"""
    date_int = int(date)
    
    # 공시날짜 행에서 해당 날짜보다 가장 가까운 과거 분기 찾기
    disclosure_row = financial_data[financial_data['계정과목'] == '공시날짜']
    if disclosure_row.empty:
        return None
    
    # 날짜 컬럼들 확인
    date_columns = [col for col in financial_data.columns if re.match(r'^\d{4}_\d+Q$', str(col))]
    
    # 해당 날짜 이전의 가장 최근 분기 찾기
    for col in reversed(date_columns):  # 최신 분기부터 역순으로
        try:
            disclosure_date = int(disclosure_row[col].iloc[0])
            if date_int >= disclosure_date:
                return col
        except:
            continue
    
    # 찾지 못하면 가장 오래된 분기
    return date_columns[0] if date_columns else None

def calculate_ratios(financial_data, quarter, shares_outstanding, stock_price):
    """재무비율 계산"""
    if not quarter:
        return {key: None for key in ['PER', 'PBR', 'ROE', 'ROA', '영업이익률', '부채비율']}
    
    # 연간화 팩터
    annualization_factors = {
        '1Q': 4, '2Q': 2, '3Q': 4/3, '4Q': 1
    }
    factor = annualization_factors.get(quarter[-2:], 1)
    
    try:
        # 재무 데이터 추출
        net_income_raw = get_account_value_by_name(financial_data, '당기순이익', quarter)
        total_assets = get_account_value_by_name(financial_data, '자산총계', quarter)
        total_equity = get_account_value_by_name(financial_data, '자본총계', quarter)
        total_debt = get_account_value_by_name(financial_data, '부채총계', quarter)
        revenue_raw = get_account_value_by_name(financial_data, '매출액', quarter)
        operating_income_raw = get_account_value_by_name(financial_data, '영업이익', quarter)
        
        # 연간화
        net_income = net_income_raw * factor
        revenue = revenue_raw * factor
        operating_income = operating_income_raw * factor
        
        # 주당 지표
        eps = net_income / shares_outstanding if shares_outstanding > 0 else 0
        bps = total_equity / shares_outstanding if shares_outstanding > 0 else 0
        
        # 비율 계산
        ratios = {
            'PER': stock_price / eps if eps > 0 else None,
            'PBR': stock_price / bps if bps > 0 else None,
            'ROE': (net_income / total_equity * 100) if total_equity > 0 else None,
            'ROA': (net_income / total_assets * 100) if total_assets > 0 else None,
            '영업이익률': (operating_income_raw / revenue_raw * 100) if revenue_raw > 0 else None,
            '부채비율': (total_debt / total_equity * 100) if total_equity > 0 else None
        }
        
        return ratios
        
    except Exception as e:
        print(f"비율 계산 오류: {e}")
        return {key: None for key in ['PER', 'PBR', 'ROE', 'ROA', '영업이익률', '부채비율']}

def process_company_analysis(row_data, result_dict):
    """단일 기업의 종가 + 재무비율을 Long Format으로 생성"""
    try:
        corp_code = row_data['고유번호']
        corp_name = row_data['종목명']
        shares_outstanding = row_data['상장주식수']
        
        # result_dict에서 재무데이터 가져오기
        financial_data = result_dict.get(corp_name)
        if financial_data is None:
            print(f"재무데이터 없음: {corp_name}")
            return []
        
        # 날짜 컬럼들 찾기 (주가 데이터)
        date_columns = [col for col in row_data.index if re.match(r'^202\d{5}$', str(col))]
        
        # 결과 리스트
        analysis_rows = []
        
        # 1. 종가 행 추가
        stock_price_row = {'종목명': corp_name, '구분': '종가'}
        for date_col in date_columns:
            stock_price = row_data[date_col]
            stock_price_row[date_col] = stock_price if pd.notna(stock_price) else None
        analysis_rows.append(stock_price_row)
        
        # 2. 각 재무비율 행 추가
        ratio_names = ['PER', 'PBR', 'ROE', 'ROA', '영업이익률', '부채비율']
        
        # 각 날짜별로 재무비율 계산
        daily_ratios = {}
        for date_col in date_columns:
            stock_price = row_data[date_col]
            if pd.notna(stock_price) and stock_price > 0:
                quarter = select_quarter_column(financial_data, date_col)
                ratios = calculate_ratios(financial_data, quarter, shares_outstanding, stock_price)
                daily_ratios[date_col] = ratios
        
        # 각 비율별로 행 생성
        for ratio_name in ratio_names:
            ratio_row = {'종목명': corp_name, '구분': ratio_name}
            for date_col in date_columns:
                if date_col in daily_ratios:
                    ratio_row[date_col] = daily_ratios[date_col].get(ratio_name)
                else:
                    ratio_row[date_col] = None
            analysis_rows.append(ratio_row)
        
        return analysis_rows
        
    except Exception as e:
        print(f"기업 {row_data.get('종목명', 'Unknown')} 처리 오류: {e}")
        return []

def create_company_analysis_dataframe(sample_data, result_dict, max_workers=5):
    """모든 기업의 분석 데이터를 Long Format DataFrame으로 생성"""
    
    all_analysis_rows = []
    
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        # 각 행(기업)을 병렬로 처리
        futures = []
        for idx, row in sample_data.iterrows():
            future = executor.submit(process_company_analysis, row, result_dict)
            futures.append(future)
        
        # 결과 수집
        for future in tqdm(futures, desc="기업별 분석 데이터 생성"):
            company_rows = future.result()
            all_analysis_rows.extend(company_rows)
    
    # DataFrame 생성
    analysis_df = pd.DataFrame(all_analysis_rows)
    
    # 컬럼 순서 정렬 (종목명, 구분, 날짜들...)
    if not analysis_df.empty:
        date_cols = [col for col in analysis_df.columns if re.match(r'^202\d{5}$', str(col))]
        date_cols.sort()
        
        column_order = ['종목명', '구분'] + date_cols
        analysis_df = analysis_df[column_order]
    
    return analysis_df


# 특정 기업만 필터링
def filter_company_analysis(analysis_df, company_name):
    """특정 기업의 분석 데이터만 추출"""
    return analysis_df[analysis_df['종목명'] == company_name].reset_index(drop=True)

# 특정 지표만 필터링  
def filter_metric_analysis(analysis_df, metric_name):
    """특정 지표의 모든 기업 데이터 추출"""
    return analysis_df[analysis_df['구분'] == metric_name].reset_index(drop=True)



In [ ]:
# 사용 예시:
# samsung_analysis = filter_company_analysis(analysis_df, '삼성전자')
# per_analysis = filter_metric_analysis(analysis_df, 'PER')

# 사용법:
# analysis_df = create_company_analysis_dataframe(sample, result_dict)
# print(analysis_df.head(20))  # 처음 20행 확인


### 2) 데이터 검증 및 수정

In [269]:
analysis_df[(analysis_df == 0).any(axis=1)]['종목명'].unique()

array(['녹십자홀딩스', '대신증권', 'CJ CGV', '보령', '이수스페셜티케미컬', 'HLB생명과학',
       '에스디바이오센서'], dtype=object)

In [ ]:
for corp in analysis_df[(analysis_df == 0).any(axis=1)]['종목명'].unique():
  unique_corp_no = final_summary[final_summary['종목명'] == corp]['고유번호'].iloc[0]
  result_dict[corp] = get_corp_quarter_data(unique_corp_no, "2024")

### 3) 지표 상관 관계 분석

# =================================================================

In [ ]:
def get_corp_idx(corp_code: str, bsns_year: str, reprt_code: str, idx_cl_code: str):
  url = "https://opendart.fss.or.kr/api/fnlttCmpnyIndx.json"

  params = {
      "crtfc_key": "4758096e577a50f35eaec98b171e463367c7c23f",
      "corp_code": corp_code,
      "bsns_year": bsns_year,
      "reprt_code": reprt_code,
      "idx_cl_code": idx_cl_code
  }

  response = requests.get(url, params=params)
  if response.status_code == 200:
    data = response.json()
    return pd.DataFrame(data['list'])
  else:
    return None

In [ ]:
def get_last_4_quarters():
    current_date = datetime.now()
    year = current_date.year
    month = current_date.month
    
    quarters = []
    
    # 현재 달을 기준으로 가장 최근 제출된 보고서 결정
    if month >= 4 and month < 6:  # 4-5월: 1분기 보고서 제출 시기
        quarters = [
            (year, '11013'),      # 1분기
            (year-1, '11011'),    # 전년 사업
            (year-1, '11014'),    # 전년 3분기
            (year-1, '11012'),    # 전년 반기
        ]
    elif month >= 6 and month < 9:  # 6-8월: 반기 보고서 제출 시기
        quarters = [
            (year, '11012'),      # 반기
            (year, '11013'),      # 1분기
            (year-1, '11011'),    # 전년 사업
            (year-1, '11014'),    # 전년 3분기
        ]
    elif month >= 9 and month < 12:  # 9-11월: 3분기 보고서 제출 시기
        quarters = [
            (year, '11014'),      # 3분기
            (year, '11012'),      # 반기
            (year, '11013'),      # 1분기
            (year-1, '11011'),    # 전년 사업
        ]
    else:  # 12월-3월: 사업 보고서 제출 시기
        # 1-3월인 경우 전년도 사업보고서가 아직 안나왔으므로 그 전년도 기준
        target_year = year - 1 if month <= 3 else year
        quarters = [
            (target_year, '11011'),    # 사업
            (target_year, '11014'),    # 3분기
            (target_year, '11012'),    # 반기
            (target_year, '11013'),    # 1분기
        ]
    
    # (연도, 보고서코드, 분기명) 형태로 반환
    result = []
    for year, code in quarters:
        quarter_name = {
            '11013': '1분기',
            '11012': '반기',
            '11014': '3분기',
            '11011': '사업'
        }[code]
        result.append({
            'year': str(year),
            'reprt_code': code,
            'quarter_name': quarter_name
        })
    
    return result

======================================================================================================================================================================================

In [ ]:
url = "https://opendart.fss.or.kr/api/fnlttSinglAcntAll.json"

params = {
    "crtfc_key": "4758096e577a50f35eaec98b171e463367c7c23f",
    "corp_code": "00362238",
    "bsns_year": "2024",
    "reprt_code": "11013",
    "fs_div": "OFS"
}

response = requests.get(url, params=params)
data = response.json()

# 전체 DataFrame
df_all = pd.DataFrame(data['list'])

# 재무제표별로 분리
df_bs = df_all[df_all['sj_div'] == 'BS'].copy()    # 재무상태표
df_is = df_all[df_all['sj_div'] == 'IS'].copy()    # 손익계산서
df_cis = df_all[df_all['sj_div'] == 'CIS'].copy()  # 포괄손익계산서

In [223]:
url = "https://opendart.fss.or.kr/api/fnlttCmpnyIndx.json"

params = {
    "crtfc_key": "4758096e577a50f35eaec98b171e463367c7c23f",
    "corp_code": "00362238",
    "bsns_year": "2024",
    "reprt_code": "11013",
    "idx_cl_code": "M210000"
}

response = requests.get(url, params=params)
response.json()
pd.DataFrame(response.json()['list'])

{'status': '000',
 'message': '정상',
 'list': [{'reprt_code': '11013',
   'bsns_year': '2024',
   'corp_code': '00362238',
   'stock_code': '079980',
   'stlm_dt': '2024-03-31',
   'idx_cl_code': 'M210000',
   'idx_cl_nm': '수익성지표',
   'idx_code': 'M211100',
   'idx_nm': '세전계속사업이익률'},
  {'reprt_code': '11013',
   'bsns_year': '2024',
   'corp_code': '00362238',
   'stock_code': '079980',
   'stlm_dt': '2024-03-31',
   'idx_cl_code': 'M210000',
   'idx_cl_nm': '수익성지표',
   'idx_code': 'M211200',
   'idx_nm': '순이익률',
   'idx_val': '-5.591'},
  {'reprt_code': '11013',
   'bsns_year': '2024',
   'corp_code': '00362238',
   'stock_code': '079980',
   'stlm_dt': '2024-03-31',
   'idx_cl_code': 'M210000',
   'idx_cl_nm': '수익성지표',
   'idx_code': 'M211250',
   'idx_nm': '총포괄이익률',
   'idx_val': '-4.784'},
  {'reprt_code': '11013',
   'bsns_year': '2024',
   'corp_code': '00362238',
   'stock_code': '079980',
   'stlm_dt': '2024-03-31',
   'idx_cl_code': 'M210000',
   'idx_cl_nm': '수익성지표',
   'idx_c

In [215]:
url = "https://opendart.fss.or.kr/api/fnlttSinglAcntAll.json"
params = {
    "crtfc_key": "4758096e577a50f35eaec98b171e463367c7c23f",
    "corp_code": "00362238",
    "bsns_year": "2024",
    "reprt_code": "11013",
    "fs_div": "OFS"
}

response = requests.get(url, params=params)
data = response.json()

# 전체 DataFrame
df_all = pd.DataFrame(data['list'])

# 재무제표별로 분리
df_bs = df_all[df_all['sj_div'] == 'BS'].copy()    # 재무상태표
df_is = df_all[df_all['sj_div'] == 'IS'].copy()    # 손익계산서
df_cis = df_all[df_all['sj_div'] == 'CIS'].copy()  # 포괄손익계산서
df_cf = df_all[df_all['sj_div'] == 'CF'].copy()    # 현금흐름표
df_sce = df_all[df_all['sj_div'] == 'SCE'].copy()  # 자본변동표

# 각 재무제표 확인
print("재무상태표 (BS):", df_bs.shape)
print("손익계산서 (IS):", df_is.shape)
print("포괄손익계산서 (CIS):", df_cis.shape)
print("현금흐름표 (CF):", df_cf.shape)
print("자본변동표 (SCE):", df_sce.shape)

# 개별 확인
df_bs.head()
df_cis.head()

재무상태표 (BS): (33, 19)
손익계산서 (IS): (0, 19)
포괄손익계산서 (CIS): (17, 19)
현금흐름표 (CF): (57, 19)
자본변동표 (SCE): (14, 19)


,rcept_no,reprt_code,bsns_year,corp_code,sj_div,sj_nm,account_id,account_nm,account_detail,thstrm_nm,thstrm_amount,frmtrm_nm,frmtrm_amount,ord,currency,thstrm_add_amount,frmtrm_q_nm,frmtrm_q_amount,frmtrm_add_amount
33,20240516000330,11013,2024,00362238,CIS,포괄손익계산서,dart_OperatingIncomeLoss,영업이익(손실),-,제 25 기 1분기,-7419068303,NaN,NaN,6,KRW,-7419068303,제 24 기 1분기,-19815804392,-19815804392
34,20240516000330,11013,2024,00362238,CIS,포괄손익계산서,dart_OtherGains,기타이익,-,제 25 기 1분기,4721900811,NaN,NaN,7,KRW,4721900811,제 24 기 1분기,5041331233,5041331233
35,20240516000330,11013,2024,00362238,CIS,포괄손익계산서,dart_OtherLosses,기타손실,-,제 25 기 1분기,4594519418,NaN,NaN,8,KRW,4594519418,제 24 기 1분기,3695647428,3695647428
36,20240516000330,11013,2024,00362238,CIS,포괄손익계산서,dart_TotalSellingGeneralAdministrativeExpenses,판매비와관리비,-,제 25 기 1분기,12457575051,NaN,NaN,9,KRW,12457575051,제 24 기 1분기,15391135965,15391135965
37,20240516000330,11013,2024,00362238,CIS,포괄손익계산서,ifrs-full_ComprehensiveIncome,총포괄손익,-,제 25 기 1분기,-10794947338,NaN,NaN,10,KRW,-10794947338,제 24 기 1분기,-18003400424,-18003400424


In [27]:
import pandas as pd
import json

url = "https://opendart.fss.or.kr/api/fnlttSinglAcntAll.json"
params = {
    "crtfc_key": "4758096e577a50f35eaec98b171e463367c7c23f",
    "corp_code": "00362238",
    "bsns_year": "2024",
    "reprt_code": "11013",
    "fs_div": "OFS"
}

response = requests.get(url, params=params)
data = response.json()

# DataFrame 생성
df = pd.DataFrame(data['list'])

# 1. 기본 데이터 정리
def clean_dart_data(df):
    """DART 재무데이터 기본 정리"""
    
    # 금액 컬럼들을 숫자로 변환
    amount_columns = ['thstrm_amount', 'frmtrm_amount', 'thstrm_add_amount', 'frmtrm_add_amount']
    for col in amount_columns:
        if col in df.columns:
            df[col] = pd.to_numeric(df[col], errors='coerce')
    
    # ord 컬럼도 숫자로 변환
    df['ord'] = pd.to_numeric(df['ord'], errors='coerce')
    
    # account_detail에서 '-' 제거
    df['account_detail'] = df['account_detail'].replace('-', None)
    
    return df

# 2. 재무제표별로 분리
def separate_by_statement(df):
    """재무제표별로 데이터 분리"""
    
    statements = {
        'BS': df[df['sj_div'] == 'BS'].copy(),      # 재무상태표
        'CIS': df[df['sj_div'] == 'CIS'].copy(),   # 포괄손익계산서  
        'CF': df[df['sj_div'] == 'CF'].copy(),     # 현금흐름표
        'SCE': df[df['sj_div'] == 'SCE'].copy()    # 자본변동표
    }
    
    return statements

# 3. 주요 계정과목별로 정리
def extract_key_accounts(statements):
    """주요 계정과목 추출"""
    
    bs = statements['BS']
    cis = statements['CIS']
    
    # 재무상태표 주요 항목
    key_bs_accounts = {
        'total_assets': bs[bs['account_nm'] == '자산총계'],
        'total_equity': bs[bs['account_nm'] == '자본총계'], 
        'total_liabilities': bs[bs['account_nm'] == '부채총계'],
        'current_assets': bs[bs['account_nm'] == '유동자산'],
        'current_liabilities': bs[bs['account_nm'] == '유동부채']
    }
    
    # 손익계산서 주요 항목
    key_cis_accounts = {
        'revenue': cis[cis['account_nm'] == '수익(매출액)'],
        'operating_income': cis[cis['account_nm'] == '영업이익(손실)'],
        'net_income': cis[cis['account_nm'] == '당기순이익(손실)'],
        'gross_profit': cis[cis['account_nm'] == '매출총이익'],
        'cost_of_sales': cis[cis['account_nm'] == '매출원가']
    }
    
    return key_bs_accounts, key_cis_accounts

# 4. 재무비율 계산을 위한 데이터 준비
def prepare_for_ratios(key_bs_accounts, key_cis_accounts):
    """재무비율 계산용 데이터 준비"""
    
    # 주요 값들을 딕셔너리로 정리
    financial_data = {}
    
    # 재무상태표 데이터
    for name, df_item in key_bs_accounts.items():
        if not df_item.empty:
            financial_data[name] = {
                'current_period': df_item.iloc[0]['thstrm_amount'],
                'previous_period': df_item.iloc[0]['frmtrm_amount']
            }
    
    # 손익계산서 데이터 (분기 누적값이므로 주의)
    for name, df_item in key_cis_accounts.items():
        if not df_item.empty:
            financial_data[name] = {
                'current_quarter': df_item.iloc[0]['thstrm_amount'],  # 1분기 누적
                'previous_quarter': df_item.iloc[0]['frmtrm_q_amount']  # 전년 동기
            }
    
    return financial_data

# 5. 실제 사용 예시
def process_dart_data(json_data):
    """전체 처리 파이프라인"""
    
    # DataFrame 생성
    df = pd.DataFrame(json_data['list'])
    
    # 기본 정리
    df = clean_dart_data(df)
    
    # 재무제표별 분리
    statements = separate_by_statement(df)
    
    # 주요 계정 추출
    key_bs, key_cis = extract_key_accounts(statements)
    
    # 재무비율 계산용 데이터 준비
    financial_data = prepare_for_ratios(key_bs, key_cis)
    
    return {
        'raw_df': df,
        'statements': statements,
        'key_accounts': {'BS': key_bs, 'CIS': key_cis},
        'financial_data': financial_data
    }

# 6. 재무비율 계산 함수들
def calculate_basic_ratios(financial_data, stock_price, shares_outstanding):
    """기본 재무비율 계산"""
    
    ratios = {}
    
    # EPS (주당순이익) - 1분기이므로 연간화 필요시 x4
    if 'net_income' in financial_data:
        eps = financial_data['net_income']['current_quarter'] / shares_outstanding
        ratios['EPS'] = eps
        
        # PER (주가수익비율)
        if eps != 0:
            ratios['PER'] = stock_price / eps
    
    # BPS (주당순자산)
    if 'total_equity' in financial_data:
        bps = financial_data['total_equity']['current_period'] / shares_outstanding
        ratios['BPS'] = bps
        
        # PBR (주가순자산비율)
        if bps != 0:
            ratios['PBR'] = stock_price / bps
    
    # ROE (자기자본이익률) - 연간화 기준
    if 'net_income' in financial_data and 'total_equity' in financial_data:
        annual_net_income = financial_data['net_income']['current_quarter'] * 4  # 1분기 -> 연간
        avg_equity = financial_data['total_equity']['current_period']
        if avg_equity != 0:
            ratios['ROE'] = (annual_net_income / avg_equity) * 100
    
    return ratios

# 사용 예시

# 실제 사용할 때:
result = process_dart_data(data)

# 재무상태표만 보기
bs_df = result['statements']['BS']

# 주요 계정 확인
print("자산총계:", result['financial_data']['total_assets'])
print("당기순이익:", result['financial_data']['net_income'])

# 재무비율 계산
ratios = calculate_basic_ratios(
    result['financial_data'], 
    stock_price=50000,  # 주가
    shares_outstanding=100000000  # 발행주식수
)
print("계산된 비율들:", ratios)

자산총계: {'current_period': np.int64(681217063045), 'previous_period': np.float64(695029639189.0)}
당기순이익: {'current_quarter': np.int64(-11055900380), 'previous_quarter': '-17709126535'}
계산된 비율들: {'EPS': np.float64(-110.5590038), 'PER': np.float64(-452.24720087429006), 'BPS': np.float64(2416.8414712), 'PBR': np.float64(20.68815873768262), 'ROE': np.float64(-18.29809776395565)}


In [41]:
data['list']

[{'rcept_no': '20240516001219',
  'reprt_code': '11013',
  'bsns_year': '2024',
  'corp_code': '01144028',
  'sj_div': 'BS',
  'sj_nm': '재무상태표',
  'account_id': 'ifrs-full_Assets',
  'account_nm': '자산총계',
  'account_detail': '-',
  'thstrm_nm': '제 24 기 1분기말',
  'thstrm_amount': '194981727230',
  'frmtrm_nm': '제 23 기말',
  'frmtrm_amount': '194913659016',
  'ord': '7',
  'currency': 'KRW'},
 {'rcept_no': '20240516001219',
  'reprt_code': '11013',
  'bsns_year': '2024',
  'corp_code': '01144028',
  'sj_div': 'BS',
  'sj_nm': '재무상태표',
  'account_id': 'ifrs-full_CurrentAssets',
  'account_nm': '유동자산',
  'account_detail': '-',
  'thstrm_nm': '제 24 기 1분기말',
  'thstrm_amount': '74221831975',
  'frmtrm_nm': '제 23 기말',
  'frmtrm_amount': '80490468477',
  'ord': '8',
  'currency': 'KRW'},
 {'rcept_no': '20240516001219',
  'reprt_code': '11013',
  'bsns_year': '2024',
  'corp_code': '01144028',
  'sj_div': 'BS',
  'sj_nm': '재무상태표',
  'account_id': 'dart_CurrentDerivativeAsset',
  'account_nm': '유동

In [33]:
url = "https://opendart.fss.or.kr/api/fnlttSinglAcntAll.json"
params = {
    "crtfc_key": "4758096e577a50f35eaec98b171e463367c7c23f",
    "corp_code": "01144028",
    "bsns_year": "2024",
    "reprt_code": "11013",
    "fs_div": "OFS"
}

response = requests.get(url, params=params)
data = response.json()

In [144]:
url = "https://opendart.fss.or.kr/api/fnlttSinglAcntAll.json"
params = {
    "crtfc_key": "4758096e577a50f35eaec98b171e463367c7c23f",
    "corp_code": "00117629",
    "bsns_year": "2024",
    "reprt_code": "11011",
    "fs_div": "OFS"
}

response = requests.get(url, params=params)
response.json()

{'status': '000',
 'message': '정상',
 'list': [{'rcept_no': '20250317000788',
   'reprt_code': '11011',
   'bsns_year': '2024',
   'corp_code': '00117629',
   'sj_div': 'BS',
   'sj_nm': '재무상태표',
   'account_id': 'ifrs-full_Assets',
   'account_nm': '자산총계',
   'account_detail': '-',
   'thstrm_nm': '제 53 기',
   'thstrm_amount': '169799432733',
   'frmtrm_nm': '제 52 기',
   'frmtrm_amount': '188999263743',
   'bfefrmtrm_nm': '제 51 기',
   'bfefrmtrm_amount': '163299828307',
   'ord': '7',
   'currency': 'KRW'},
  {'rcept_no': '20250317000788',
   'reprt_code': '11011',
   'bsns_year': '2024',
   'corp_code': '00117629',
   'sj_div': 'BS',
   'sj_nm': '재무상태표',
   'account_id': 'ifrs-full_CurrentAssets',
   'account_nm': '유동자산',
   'account_detail': '-',
   'thstrm_nm': '제 53 기',
   'thstrm_amount': '91589796456',
   'frmtrm_nm': '제 52 기',
   'frmtrm_amount': '117140081457',
   'bfefrmtrm_nm': '제 51 기',
   'bfefrmtrm_amount': '84175209329',
   'ord': '8',
   'currency': 'KRW'},
  {'rcept_no'

In [64]:
df = pd.DataFrame(response.json()['list'])
revenue_accounts = df[df['account_nm'].str.contains('매출', na=False)]['account_nm'].unique()
print("매출 관련 계정과목들:", revenue_accounts)

매출 관련 계정과목들: ['매출채권 및 기타채권' '장기매출채권 및 기타채권' '매출원가' '매출총이익' '매출액']


In [73]:
import pandas as pd

def api_call(reprt_code):
    url = "https://opendart.fss.or.kr/api/fnlttSinglAcntAll.json"
    params = {
        "crtfc_key": "4758096e577a50f35eaec98b171e463367c7c23f",
        "corp_code": "00117629",
        "bsns_year": "2024",
        "reprt_code": reprt_code,
        "fs_div": "OFS"
    }

    response = requests.get(url, params=params)
    return response

def get_quarter_data(response):
    """한 분기 데이터를 딕셔너리로 반환"""
    df = pd.DataFrame(response.json()['list'])
    
    accounts_needed = {
    'CIS': [
    '매출액', '매출총이익', '매출원가', '영업이익(손실)',
    '당기순이익(손실)', '금융원가', '금융수익'
    ],
    'BS_자산': [
        '자산총계', '유동자산', '현금및현금성자산'
    ],
    'BS_부채': [
        '부채총계', '유동부채'
    ],
    'BS_자본': [
        '자본총계', '자본금'
    ]
    }   
    
    quarter_data = {}
    
    for category, account_list in accounts_needed.items():
        for account in account_list:
            found = df[df['account_nm'] == account]
            if not found.empty:
                item = found.iloc[0]
                key = f"{category}_{account}"
                
                # 빈 문자열과 NaN 모두 처리
                amount = item['thstrm_amount']
                if pd.notna(amount) and amount != '':
                    quarter_data[key] = int(amount)
                else:
                    quarter_data[key] = 0
            else:
                key = f"{category}_{account}"
                quarter_data[key] = 0
    
    return quarter_data

# 4개 분기 API 호출 및 데이터 수집
response_2024_1q = api_call("11013")  # 실제 API 호출
response_2024_2q = api_call("11012")  
response_2024_3q = api_call("11014")  
response_2024_4q = api_call("11011")

q1_2024 = get_quarter_data(response_2024_1q)
q2_2024 = get_quarter_data(response_2024_2q)
q3_2024 = get_quarter_data(response_2024_3q)
q4_2024 = get_quarter_data(response_2024_4q)

# 통합 테이블 생성
results = []

accounts_needed = {
    'CIS': [
        '매출액', '매출총이익', '매출원가', '영업이익(손실)',
        '당기순이익(손실)', '금융원가', '금융수익'
    ],
    'BS_자산': [
        '자산총계', '유동자산', '현금및현금성자산'
    ],
    'BS_부채': [
        '부채총계', '유동부채'
    ],
    'BS_자본': [
        '자본총계', '자본금'
    ]
}

for category, account_list in accounts_needed.items():
    for account in account_list:
        key = f"{category}_{account}"
        
        results.append({
            '구분': category,
            '계정과목': account,
            '2024_1Q': q1_2024.get(key, 0),
            '2024_2Q': q2_2024.get(key, 0),
            '2024_3Q': q3_2024.get(key, 0),
            '2024_4Q': q4_2024.get(key, 0),
            '발견여부': 'O' if q1_2024.get(key, 0) != 0 else 'X'
        })

# DataFrame 생성 및 출력
result_df = pd.DataFrame(results)

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
print(result_df)

# CSV 저장
result_df.to_csv('4분기_통합_재무데이터.csv', index=False, encoding='utf-8-sig')
print("\n4분기_통합_재무데이터.csv 파일로 저장되었습니다.")

       구분       계정과목       2024_1Q       2024_2Q       2024_3Q       2024_4Q  \
0     CIS        매출액   59818207275   52423785775   38672870430  200428941393   
1     CIS      매출총이익    3932677013    4510855616    3657530292   15685512612   
2     CIS       매출원가   55885530262   47912930159   35015340138  184743428781   
3     CIS   영업이익(손실)    1737986255    1729655693    1296369670    4842957555   
4     CIS  당기순이익(손실)     428035612  -22043925893     424321505  -21886954872   
5     CIS       금융원가             0             0             0             0   
6     CIS       금융수익     383091040    3672174179    4574458276    1683198592   
7   BS_자산       자산총계  200441928999  171432098377  167467997955  169799432733   
8   BS_자산       유동자산  130182304534  102309394683   94756690768   91589796456   
9   BS_자산   현금및현금성자산    3741167268   13532517806   26663537145   28346004622   
10  BS_부채       부채총계  117761966963   79832104518   64475313554   68912141746   
11  BS_부채       유동부채  114576633613   764

In [249]:
import pandas as pd
import re
from concurrent.futures import ThreadPoolExecutor
from tqdm import tqdm

def get_account_value_by_name(financial_data, account_name, quarter_column):
    """계정과목 이름으로 값 찾기"""
    mask = financial_data['계정과목'] == account_name
    account_row = financial_data[mask]
    
    if not account_row.empty:
        try:
            return float(account_row[quarter_column].iloc[0])
        except:
            return 0
    return 0

def select_quarter_column(financial_data, date):
    """해당 날짜에 맞는 분기 컬럼 선택"""
    date_int = int(date)
    
    # 공시날짜 행에서 해당 날짜보다 가장 가까운 과거 분기 찾기
    disclosure_row = financial_data[financial_data['계정과목'] == '공시날짜']
    if disclosure_row.empty:
        return None
    
    # 날짜 컬럼들 확인
    date_columns = [col for col in financial_data.columns if re.match(r'^\d{4}_\d+Q$', str(col))]
    
    # 해당 날짜 이전의 가장 최근 분기 찾기
    for col in reversed(date_columns):  # 최신 분기부터 역순으로
        try:
            disclosure_date = int(disclosure_row[col].iloc[0])
            if date_int >= disclosure_date:
                return col
        except:
            continue
    
    # 찾지 못하면 가장 오래된 분기
    return date_columns[0] if date_columns else None

def calculate_ratios(financial_data, quarter, shares_outstanding, stock_price):
    """재무비율 계산"""
    if not quarter:
        return {key: None for key in ['PER', 'PBR', 'ROE', 'ROA', '영업이익률', '부채비율']}
    
    # 연간화 팩터
    annualization_factors = {
        '1Q': 4, '2Q': 2, '3Q': 4/3, '4Q': 1
    }
    factor = annualization_factors.get(quarter[-2:], 1)
    
    try:
        # 재무 데이터 추출
        net_income_raw = get_account_value_by_name(financial_data, '당기순이익', quarter)
        total_assets = get_account_value_by_name(financial_data, '자산총계', quarter)
        total_equity = get_account_value_by_name(financial_data, '자본총계', quarter)
        total_debt = get_account_value_by_name(financial_data, '부채총계', quarter)
        revenue_raw = get_account_value_by_name(financial_data, '매출액', quarter)
        operating_income_raw = get_account_value_by_name(financial_data, '영업이익', quarter)
        
        # 연간화
        net_income = net_income_raw * factor
        revenue = revenue_raw * factor
        operating_income = operating_income_raw * factor
        
        # 주당 지표
        eps = net_income / shares_outstanding if shares_outstanding > 0 else 0
        bps = total_equity / shares_outstanding if shares_outstanding > 0 else 0
        
        # 비율 계산
        ratios = {
            'PER': stock_price / eps if eps > 0 else None,
            'PBR': stock_price / bps if bps > 0 else None,
            'ROE': (net_income / total_equity * 100) if total_equity > 0 else None,
            'ROA': (net_income / total_assets * 100) if total_assets > 0 else None,
            '영업이익률': (operating_income_raw / revenue_raw * 100) if revenue_raw > 0 else None,
            '부채비율': (total_debt / total_equity * 100) if total_equity > 0 else None
        }
        
        return ratios
        
    except Exception as e:
        print(f"비율 계산 오류: {e}")
        return {key: None for key in ['PER', 'PBR', 'ROE', 'ROA', '영업이익률', '부채비율']}

def process_company_analysis(row_data, result_dict):
    """단일 기업의 종가 + 재무비율을 Long Format으로 생성"""
    try:
        corp_code = row_data['고유번호']
        corp_name = row_data['종목명']
        shares_outstanding = row_data['상장주식수']
        
        # result_dict에서 재무데이터 가져오기
        financial_data = result_dict.get(corp_name)
        if financial_data is None:
            print(f"재무데이터 없음: {corp_name}")
            return []
        
        # 날짜 컬럼들 찾기 (주가 데이터)
        date_columns = [col for col in row_data.index if re.match(r'^202\d{5}$', str(col))]
        
        # 결과 리스트
        analysis_rows = []
        
        # 1. 종가 행 추가
        stock_price_row = {'종목명': corp_name, '구분': '종가'}
        for date_col in date_columns:
            stock_price = row_data[date_col]
            stock_price_row[date_col] = stock_price if pd.notna(stock_price) else None
        analysis_rows.append(stock_price_row)
        
        # 2. 각 재무비율 행 추가
        ratio_names = ['PER', 'PBR', 'ROE', 'ROA', '영업이익률', '부채비율']
        
        # 각 날짜별로 재무비율 계산
        daily_ratios = {}
        for date_col in date_columns:
            stock_price = row_data[date_col]
            if pd.notna(stock_price) and stock_price > 0:
                quarter = select_quarter_column(financial_data, date_col)
                ratios = calculate_ratios(financial_data, quarter, shares_outstanding, stock_price)
                daily_ratios[date_col] = ratios
        
        # 각 비율별로 행 생성
        for ratio_name in ratio_names:
            ratio_row = {'종목명': corp_name, '구분': ratio_name}
            for date_col in date_columns:
                if date_col in daily_ratios:
                    ratio_row[date_col] = daily_ratios[date_col].get(ratio_name)
                else:
                    ratio_row[date_col] = None
            analysis_rows.append(ratio_row)
        
        return analysis_rows
        
    except Exception as e:
        print(f"기업 {row_data.get('종목명', 'Unknown')} 처리 오류: {e}")
        return []

def create_company_analysis_dataframe(sample_data, result_dict, max_workers=5):
    """모든 기업의 분석 데이터를 Long Format DataFrame으로 생성"""
    
    all_analysis_rows = []
    
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        # 각 행(기업)을 병렬로 처리
        futures = []
        for idx, row in sample_data.iterrows():
            future = executor.submit(process_company_analysis, row, result_dict)
            futures.append(future)
        
        # 결과 수집
        for future in tqdm(futures, desc="기업별 분석 데이터 생성"):
            company_rows = future.result()
            all_analysis_rows.extend(company_rows)
    
    # DataFrame 생성
    analysis_df = pd.DataFrame(all_analysis_rows)
    
    # 컬럼 순서 정렬 (종목명, 구분, 날짜들...)
    if not analysis_df.empty:
        date_cols = [col for col in analysis_df.columns if re.match(r'^202\d{5}$', str(col))]
        date_cols.sort()
        
        column_order = ['종목명', '구분'] + date_cols
        analysis_df = analysis_df[column_order]
    
    return analysis_df

# 사용법:
# analysis_df = create_company_analysis_dataframe(sample, result_dict)
# print(analysis_df.head(20))  # 처음 20행 확인

# 특정 기업만 필터링
def filter_company_analysis(analysis_df, company_name):
    """특정 기업의 분석 데이터만 추출"""
    return analysis_df[analysis_df['종목명'] == company_name].reset_index(drop=True)

# 특정 지표만 필터링  
def filter_metric_analysis(analysis_df, metric_name):
    """특정 지표의 모든 기업 데이터 추출"""
    return analysis_df[analysis_df['구분'] == metric_name].reset_index(drop=True)

# 사용 예시:
# samsung_analysis = filter_company_analysis(analysis_df, '삼성전자')
# per_analysis = filter_metric_analysis(analysis_df, 'PER')

In [250]:
analysis_df = create_company_analysis_dataframe(sample, result_dict)

기업별 분석 데이터 생성: 100%|██████████| 134/134 [00:14<00:00,  9.54it/s]


In [267]:
filter_company_analysis(analysis_df, '제이스코홀딩스').filter(items=['종목명','구분','20240102','20241230'])

,종목명,구분,20240102,20241230
0,제이스코홀딩스,종가,1430.000000,1693.000000
1,제이스코홀딩스,PER,NaN,NaN
2,제이스코홀딩스,PBR,1.400996,1.623150
3,제이스코홀딩스,ROE,-38.864932,-13.082513
4,제이스코홀딩스,ROA,-19.087926,-6.709918
5,제이스코홀딩스,영업이익률,-15.078510,-68.675153
6,제이스코홀딩스,부채비율,103.610031,94.972764


In [258]:
analysis_df

,종목명,구분,20240102,20240103,20240104,20240105,20240108,20240109,20240110,20240111,...,20241216,20241217,20241218,20241219,20241220,20241223,20241224,20241226,20241227,20241230
0,제이스코홀딩스,종가,1430.000000,1477.000000,1408.000000,1401.000000,1414.000000,1425.000000,1460.000000,1469.000000,...,1854.000000,1871.000000,1825.000000,1847.000000,1744.000000,1725.000000,1739.000000,1702.000000,1626.000000,1693.000000
1,제이스코홀딩스,PER,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,제이스코홀딩스,PBR,1.400996,1.447042,1.379442,1.372584,1.385320,1.396097,1.430387,1.439205,...,1.777508,1.793806,1.749704,1.770796,1.672046,1.653830,1.667252,1.631779,1.558914,1.623150
3,제이스코홀딩스,ROE,-38.864932,-38.864932,-38.864932,-38.864932,-38.864932,-38.864932,-38.864932,-38.864932,...,-13.082513,-13.082513,-13.082513,-13.082513,-13.082513,-13.082513,-13.082513,-13.082513,-13.082513,-13.082513
4,제이스코홀딩스,ROA,-19.087926,-19.087926,-19.087926,-19.087926,-19.087926,-19.087926,-19.087926,-19.087926,...,-6.709918,-6.709918,-6.709918,-6.709918,-6.709918,-6.709918,-6.709918,-6.709918,-6.709918,-6.709918
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
933,DGB금융지주,PBR,0.420864,0.420864,0.414858,0.407352,0.404349,0.407852,0.412356,0.412356,...,0.443487,0.425199,0.431295,0.427231,0.417579,0.428755,0.429263,0.428755,0.422659,0.415547
934,DGB금융지주,ROE,5.416470,5.416470,5.416470,5.416470,5.416470,5.416470,5.416470,5.416470,...,-1.397186,-1.397186,-1.397186,-1.397186,-1.397186,-1.397186,-1.397186,-1.397186,-1.397186,-1.397186
935,DGB금융지주,ROA,4.086800,4.086800,4.086800,4.086800,4.086800,4.086800,4.086800,4.086800,...,-1.046711,-1.046711,-1.046711,-1.046711,-1.046711,-1.046711,-1.046711,-1.046711,-1.046711,-1.046711
936,DGB금융지주,영업이익률,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
for corp in analysis_df[(analysis_df == 0).any(axis=1)]['종목명'].unique():
  unique_corp_no = final_summary[final_summary['종목명'] == corp]['고유번호'].iloc[0]
  result_dict[corp] = get_corp_quarter_data(unique_corp_no, "2024")
  


현대에버다임 00228536
신원 00136864
그린케미칼 00454946
우주일렉트로 00390903
브랜드엑스코퍼레이션 01391103
아미코젠 00545734
시너지이노베이션 00475985
흥국화재 00103176
나무가 00819374
에치에프알 01108442
화승인더 00166227
제주은행 00148832
엠아이텍 00531829
에스에이엠티 00128661
인카금융서비스 01013694
미코 00366942
한전산업 00194947
대원제약 00111999
토비스 00390365
서희건설 00219848
영진약품 00141149
현대무벡스 01358463
현대퓨처넷 00403632
큐로셀 01492651
현대그린푸드 01721809
HJ중공업 00633835
바이오노트 00622787
지노믹트리 01108248
비츠로셀 00154718
코스메카코리아 00763473
엘앤씨바이오 01181515
동국제약 00114808
녹십자홀딩스 00108135
현대지에프홀딩스 00105280
대신증권 00110893
디어유 01456011
엠로 00396925
와이지엔터테인먼트 00613318
CJ CGV 00303873
보령 00123143
이수스페셜티케미컬 01762569
LX인터내셔널 00120076
HDC현대산업개발 01310269
HLB생명과학 00365590
한국콜마 00939331
에스디바이오센서 00854997
DGB금융지주 00878915


In [259]:
final_summary[final_summary['종목명'] == '제이스코홀딩스']

,시장구분,종목명,종목코드,소속부,시총_증감률,주가_수익률,상장주식수,고유번호
0,KOSDAQ,제이스코홀딩스,023440,중견기업부,0.446489,0.201705,61598899,00148531


In [247]:
result_dict["와이지엔터테인먼트"] = get_corp_quarter_data("00613318", "2024")

In [248]:
# net_income_raw = get_account_value_by_name(financial_data, '당기순이익', quarter)
# total_assets = get_account_value_by_name(financial_data, '자산총계', quarter)
# total_equity = get_account_value_by_name(financial_data, '자본총계', quarter)
# total_debt = get_account_value_by_name(financial_data, '부채총계', quarter)
# revenue_raw = get_account_value_by_name(financial_data, '매출액', quarter)
# operating_income_raw = get_account_value_by_name(financial_data, '영업이익', quarter)

result_dict.get("와이지엔터테인먼트")

,구분,계정과목,2023_4Q,2024_1Q,2024_2Q,2024_3Q,2024_4Q,발견여부
0,공시정보,공시날짜,20240403,20240516,20240814,20241114,20250320,O
1,CIS,매출액,381504959565,44595252176,51189865240,41622518891,198017360839,O
2,CIS,매출총이익,131083246961,13338527985,14695380884,13672565928,62612521955,O
3,CIS,매출원가,250421712604,31256724191,36494484356,27949952963,135404838884,O
4,CIS,영업이익,58051670284,-6707357201,-8814003756,-4614787785,-21506424280,O
5,CIS,당기순이익,53009537215,-2460406407,-4555831012,-5485153155,1293830821,O
6,CIS,금융원가,-8752248531,-1188514224,-170895495,67866737,3197546448,O
7,CIS,금융수익,13216214291,6793502787,7307067780,-3622716114,13353485640,O
8,BS_자산,자산총계,544577306410,516199884257,500874992629,493493255370,508053781011,O
9,BS_자산,유동자산,174732951285,137824359958,140190939190,130967370337,157783861770,O


In [241]:
# 1분기보고서 : 11013
# 반기보고서 : 11012
# 3분기보고서 : 11014
# 사업보고서 : 11011

res =api_call("00613318", "2024", "11011").json()
res

{'status': '000',
 'message': '정상',
 'list': [{'rcept_no': '20250320001675',
   'reprt_code': '11011',
   'bsns_year': '2024',
   'corp_code': '00613318',
   'sj_div': 'BS',
   'sj_nm': '재무상태표',
   'account_id': 'ifrs-full_Assets',
   'account_nm': '자산총계',
   'account_detail': '-',
   'thstrm_nm': '제 27 기',
   'thstrm_amount': '508053781011',
   'frmtrm_nm': '제 26 기',
   'frmtrm_amount': '544577306410',
   'bfefrmtrm_nm': '제 25 기',
   'bfefrmtrm_amount': '501420910066',
   'ord': '7',
   'currency': 'KRW'},
  {'rcept_no': '20250320001675',
   'reprt_code': '11011',
   'bsns_year': '2024',
   'corp_code': '00613318',
   'sj_div': 'BS',
   'sj_nm': '재무상태표',
   'account_id': 'ifrs-full_CurrentAssets',
   'account_nm': '유동자산',
   'account_detail': '-',
   'thstrm_nm': '제 27 기',
   'thstrm_amount': '157783861770',
   'frmtrm_nm': '제 26 기',
   'frmtrm_amount': '174732951285',
   'bfefrmtrm_nm': '제 25 기',
   'bfefrmtrm_amount': '188251706525',
   'ord': '8',
   'currency': 'KRW'},
  {'rcept_n

In [257]:
analysis_df[analysis_df.isnull().any(axis=1)]

,종목명,구분,20240102,20240103,20240104,20240105,20240108,20240109,20240110,20240111,...,20241216,20241217,20241218,20241219,20241220,20241223,20241224,20241226,20241227,20241230
1,제이스코홀딩스,PER,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,퀀타매트릭스,PER,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
15,삼익악기,PER,22.843962,22.843962,22.674591,22.674591,22.632248,22.674591,22.674591,22.547562,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
22,나노엔텍,PER,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1626.461548,1645.624571,1638.438437,1619.275415,1571.367858,1609.693903,1580.949369,1540.227946,1521.064923,1559.390969
43,티라유텍,PER,127.244615,127.113973,123.978582,133.776679,135.344374,135.605657,140.570026,136.128222,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
911,HLB생명과학,PER,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
925,에스디바이오센서,PER,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,13.278991,13.302493,13.231985,12.609166,12.503404,12.491652,12.350637,12.103859,11.927589,12.080356
929,에스디바이오센서,영업이익률,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
932,DGB금융지주,PER,7.770071,7.770071,7.659202,7.520616,7.465181,7.529855,7.613007,7.613007,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [251]:
analysis_df[(analysis_df == 0).any(axis=1)]['종목명'].unique()

array(['녹십자홀딩스', '대신증권', 'CJ CGV', '보령', '이수스페셜티케미컬', 'HLB생명과학',
       '에스디바이오센서'], dtype=object)